In [5]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from PIL import Image

import tensorflow as tf
import re
from keras.layers import Dense
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
import pandas as pd
import cv2
from sklearn.cluster import KMeans
from scipy.spatial import distance
import warnings
from sklearn.cluster import KMeans
import requests
from io import BytesIO
import webcolors

warnings.simplefilter(action='ignore')

In [6]:
def corrigir_caracteres_mal_codificados(texto):
  
    mapeamento_substituicao = {
        "Ã¡": "á",
        "Ã©": "é",
       
    }

  
    for chave, valor in mapeamento_substituicao.items():
        texto = texto.replace(chave, valor)

    return texto




In [7]:


def extract_colors_rgb_and_names_from_image(image_url, num_colors=5):
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))
    image = image.resize((100, 100))
    image_array = np.array(image)
    pixels = image_array.reshape(-1, 3)
    kmeans = KMeans(n_clusters=num_colors, random_state=0).fit(pixels)
    cluster_centers = kmeans.cluster_centers_.astype(int)
    
    colors_rgb = cluster_centers.tolist()
    color_names = [get_color_name(center) for center in cluster_centers]
    
    return colors_rgb, color_names

def get_color_name(rgb_value):
    min_colors = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - rgb_value[0]) ** 2
        gd = (g_c - rgb_value[1]) ** 2
        bd = (b_c - rgb_value[2]) ** 2
        min_colors[(rd + gd + bd)] = name
    return min_colors[min(min_colors.keys())]

df = pd.read_csv(r"D:\testekeras\basedados\produtos.csv", sep=",", encoding="latin-1")

image_urls = pd.melt(df, id_vars=['nome'], value_vars=['img0', 'img1'], var_name='fonte', value_name='valor')
image_urls.fillna(0, inplace=True)

num_colors = 5

lista_dicts = []
for i, row in image_urls.iterrows():
    if isinstance(row[2], str):
        dict_items = {}
      
        cores_da_imagem_rgb, nomes_das_cores = extract_colors_rgb_and_names_from_image(row[2], num_colors)
        
        dict_items['Nome'] = row[0]
        dict_items['CodigoRgb'] = cores_da_imagem_rgb
        dict_items['NomesCores'] = nomes_das_cores
        dict_items['imagem'] = row[2]
        print(dict_items)
        lista_dicts.append(dict_items)


#df_final = pd.DataFrame(lista_dicts)


#df_final['tipo'] = df_final['Nome'].map(df.set_index('nome')['tipo'])

#df_final.to_csv('dados_produtos_com_cores.csv', sep=";", encoding="latin-1", index=False)




{'Nome': 'Cozinha Compacta Iluminata 8 PT 4 GV Branca e Ã\x89bano', 'CodigoRgb': [[125, 113, 101], [206, 201, 195], [40, 36, 29], [177, 168, 160], [82, 75, 67]], 'NomesCores': ['dimgray', 'silver', 'black', 'darkgray', 'darkslategray'], 'imagem': 'https://static.mobly.com.br/p/GENIALFLEX-Cozinha-Compacta-Iluminata-8-PT-4-GV-Branca-e-C389bano-6321-2963011-1-cart.jpg'}
{'Nome': 'ArmÃ¡rio AÃ©reo Canelone com Vidro 3 PT Preto', 'CodigoRgb': [[254, 254, 254], [67, 68, 66], [30, 31, 30], [176, 176, 176], [85, 86, 84]], 'NomesCores': ['white', 'darkslategray', 'black', 'darkgray', 'dimgray'], 'imagem': 'https://static.mobly.com.br/p/Politorno-ArmC3A1rio-AC3A9reo-Canelone-com-Vidro-3-PT-Preto-5823-1819211-1-cart.jpg'}
{'Nome': 'ArmÃ¡rio AÃ©reo Evidence 3 PT Branco 120 cm', 'CodigoRgb': [[254, 254, 254], [178, 178, 178], [231, 231, 231], [200, 200, 200], [237, 237, 237]], 'NomesCores': ['white', 'darkgray', 'lavender', 'silver', 'whitesmoke'], 'imagem': 'https://static.mobly.com.br/p/Poliman-Ar

In [8]:
lista_dicts

[{'Nome': 'Cozinha Compacta Iluminata 8 PT 4 GV Branca e Ã\x89bano',
  'CodigoRgb': [[125, 113, 101],
   [206, 201, 195],
   [40, 36, 29],
   [177, 168, 160],
   [82, 75, 67]],
  'NomesCores': ['dimgray', 'silver', 'black', 'darkgray', 'darkslategray'],
  'imagem': 'https://static.mobly.com.br/p/GENIALFLEX-Cozinha-Compacta-Iluminata-8-PT-4-GV-Branca-e-C389bano-6321-2963011-1-cart.jpg'},
 {'Nome': 'ArmÃ¡rio AÃ©reo Canelone com Vidro 3 PT Preto',
  'CodigoRgb': [[254, 254, 254],
   [67, 68, 66],
   [30, 31, 30],
   [176, 176, 176],
   [85, 86, 84]],
  'NomesCores': ['white', 'darkslategray', 'black', 'darkgray', 'dimgray'],
  'imagem': 'https://static.mobly.com.br/p/Politorno-ArmC3A1rio-AC3A9reo-Canelone-com-Vidro-3-PT-Preto-5823-1819211-1-cart.jpg'},
 {'Nome': 'ArmÃ¡rio AÃ©reo Evidence 3 PT Branco 120 cm',
  'CodigoRgb': [[254, 254, 254],
   [178, 178, 178],
   [231, 231, 231],
   [200, 200, 200],
   [237, 237, 237]],
  'NomesCores': ['white', 'darkgray', 'lavender', 'silver', 'whitesmo

In [8]:
df_produtos = pd.read_csv(r"D:\testekeras\dados_finais_com_rgb.csv",sep=",")
df_produtos.fillna(0, inplace=True)
df_produtos['Nome'] = df_produtos['Nome'].apply(lambda k: corrigir_caracteres_mal_codificados(k))

In [10]:
df_produtos

,Nome,CodigoRgb,imagem,tipo
0,Cozinha Compacta Iluminata 8 PT 4 GV Branca e ...,[[125 113 101]\n [206 201 195]\n [ 40 36 29]...,https://static.mobly.com.br/p/GENIALFLEX-Cozin...,Conjunto Cozinha
1,Armário Aéreo Canelone com Vidro 3 PT Preto,[[254 254 254]\n [ 67 68 66]\n [ 30 31 30]...,https://static.mobly.com.br/p/Politorno-ArmC3A...,Outro
2,Armário Aéreo Evidence 3 PT Branco 120 cm,[[254 254 254]\n [178 178 178]\n [231 231 231]...,https://static.mobly.com.br/p/Poliman-ArmC3A1r...,Outro
3,BalcÃ£o de Cozinha Iluminata 2 PT Mint 80 cm,[[253 253 251]\n [183 174 163]\n [229 113 69]...,https://static.mobly.com.br/p/GENIALFLEX-BalcC...,Balcao de Cozinha
4,Paneleiro Maxxi 2 PT Branco,[[240 239 239]\n [200 199 199]\n [254 254 254]...,https://static.mobly.com.br/p/Kappesberg-Panel...,Paneleiro
...,...,...,...,...
221,Cozinha Compacta Com Balcao Para Pia 10027x170...,[[ 55 51 46]\n [201 197 192]\n [116 111 102]...,https://static.mobly.com.br/p/Poliman-Cozinha-...,Conjunto Cozinha
222,Armário de Cozinha Completa 325cm Rustic Emill...,[[253 253 251]\n [125 98 72]\n [190 182 171]...,https://static.mobly.com.br/p/Madesa-ArmC3A1ri...,Cozinha completa
223,Armário de Cozinha Completa Madesa Emilly Danc...,[[252 252 250]\n [102 81 65]\n [204 180 152]...,https://static.mobly.com.br/p/Madesa-ArmC3A1ri...,Cozinha completa
224,BalcÃ£o de Cozinha Evidence 3 PT 2 GV Branco,[[227 227 226]\n [168 169 167]\n [254 254 254]...,https://static.mobly.com.br/p/Poliman-BalcC3A3...,Balcao de Cozinha


In [ ]:
def extract_colors_from_image(image_url, num_colors=5):
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))
    image = image.resize((100, 100))
    image_array = np.array(image)
    pixels = image_array.reshape(-1, 3)
    kmeans = KMeans(n_clusters=num_colors, random_state=0).fit(pixels)
    cluster_centers = kmeans.cluster_centers_.astype(int)
    return cluster_centers


df_treinamento = pd.read_csv(r"D:\testekeras\dados_finais_com_rgb.csv",sep=",",encoding="latin-1")


df_avaliacao = pd.read_csv(r"D:\testekeras\avaliacao.csv",sep=";",encoding="latin-1")


X_train = []
y_train = []

for i, row in df_treinamento.iterrows():
    if isinstance(row['imagem'], str):
        cores_da_imagem = extract_colors_from_image(row['imagem'])
        atributos = cores_da_imagem.tolist()  
        X_train.append(atributos)
        y_train.append(row['tipo']) 


X_test = []
y_test = []

for i, row in df_avaliacao.iterrows():
    if isinstance(row['imagem'], str):
        cores_da_imagem = extract_colors_from_image(row['imagem'])
        atributos = cores_da_imagem.tolist() 
        X_test.append(atributos)
        y_test.append(row['tipo']) 


label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


y_train_encoded = to_categorical(y_train_encoded)
y_test_encoded = to_categorical(y_test_encoded)

model = Sequential()
model.add(Dense(128, input_dim=3, activation='relu')) 
model.add(Dense(64, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(np.array(X_train), y_train_encoded, epochs=10, batch_size=32, validation_split=0.2)


accuracy = model.evaluate(np.array(X_test), y_test_encoded)
print(f"Acurácia do modelo: {accuracy[1]}")
